In [5]:
from typing import Any, Dict, Tuple, List
from itertools import groupby

In [6]:
def compute_ttr(backlog: Dict[Tuple[Any, ...], float], *, zero_tol: float = 1e-9, return_open: bool = True) -> Dict[Tuple[Any, Any], List[Dict[str, Any]]]:
    if not backlog:
        return {}

    # Prepare (group, time_tuple, value) triples and sort by (group, time)
    triples = []
    for k, v in backlog.items():
        if len(k) < 3:
            raise ValueError(f"Key {k} must have at least (loc, com, time...).")
        triples.append(((k[0], k[1]), k[2:], float(v)))
    triples.sort(key=lambda x: (x[0], x[1]))  # lexicographic time handles multi-index

    results: Dict[Tuple[Any, Any], List[Dict[str, Any]]] = {}

    for g, grp in groupby(triples, key=lambda x: x[0]):
        episodes: List[Dict[str, Any]] = []
        in_run = False
        start = None
        peak = area = 0.0
        steps = 0

        for _, t, val in grp:
            if val > zero_tol:
                if not in_run:
                    in_run = True
                    start = t
                    peak = val
                    area = val
                    steps = 1
                else:
                    steps += 1
                    area += val
                    if val > peak:
                        peak = val
            elif in_run:
                episodes.append({
                    "start_idx": start,
                    "end_idx": t,                 # first nonpositive period
                    "duration_steps": steps,
                    "peak": peak,
                    "area": area,
                })
                in_run = False

        if in_run and return_open:
            episodes.append({
                "start_idx": start,
                "end_idx": None,                  # not yet recovered
                "duration_steps": steps,
                "peak": peak,
                "area": area,
            })

        results[g] = episodes

    return results

In [7]:
backlog = {
    ('loc1','com1_sold',0,0,0): 0.0,
    ('loc1','com1_sold',0,0,1): 2.0,
    ('loc1','com1_sold',0,0,2): 3.0,
    ('loc1','com1_sold',0,0,3): 0.0,   # recovery here
    ('loc1','com1_sold',0,0,4): 0.0,
    ('loc1','com1_sold',0,0,5): 1.0,   # second disruption
    ('loc1','com1_sold',0,0,6): 0.0,   # recover
    ('loc2','com1_sold',1,2): 0.5,     # open episode (no recovery yet)
}

out = compute_ttr(backlog)
# out[('loc1','com1_sold')] ->
# [
#   {'start_idx': (0,0,1), 'end_idx': (0,0,3), 'duration_steps': 2, 'peak': 3.0, 'area': 5.0},
#   {'start_idx': (0,0,5), 'end_idx': (0,0,6), 'duration_steps': 1, 'peak': 1.0, 'area': 1.0},
# ]
# out[('loc2','com1_sold')] ->
# [
#   {'start_idx': (1,2), 'end_idx': None, 'duration_steps': 1, 'peak': 0.5, 'area': 0.5},
# ]


In [8]:
out

{('loc1',
  'com1_sold'): [{'start_idx': (0, 0, 1),
   'end_idx': (0, 0, 3),
   'duration_steps': 2,
   'peak': 3.0,
   'area': 5.0}, {'start_idx': (0, 0, 5),
   'end_idx': (0, 0, 6),
   'duration_steps': 1,
   'peak': 1.0,
   'area': 1.0}],
 ('loc2',
  'com1_sold'): [{'start_idx': (1, 2),
   'end_idx': None,
   'duration_steps': 1,
   'peak': 0.5,
   'area': 0.5}]}